In [1]:
import torch
# Check if PyTorch supports the built-in alternative (SDPA)
print(hasattr(torch.nn.functional, 'scaled_dot_product_attention'))

True


In [2]:
from ultralytics import YOLO
# Load a YOLOv8 model (you can replace 'yolov8n.pt' with your model path)
model = YOLO('yolov8n.pt')


In [ ]:
# Train the model
results = model.train(
    data='data.yaml',  # Path to your config file
    epochs=1,         # 50 runs through the data is a good start
    imgsz=640,         # Resizes images to 640x640 pixels for training
    batch=16,          # How many images to process at once (lower if you run out of memory)
    device=0           # Use '0' for GPU, or 'cpu' if you don't have an NVIDIA GPU
)

print("Training complete!")

In [5]:
# Train the model
results = model.train(
    data='data.yaml',  # Path to your config file
    epochs=2,         # 50 runs through the data is a good start
    imgsz=640,         # Resizes images to 640x640 pixels for training
    batch=16,          # How many images to process at once (lower if you run out of memory)
    # device=0           # Use '0' for GPU, or 'cpu' if you don't have an NVIDIA GPU
    device='cpu'           # Use '0' for GPU, or 'cpu' if you don't have an NVIDIA GPU
)

print("Training complete!")

New https://pypi.org/project/ultralytics/8.3.234 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.12.12 torch-2.9.1+cpu CPU (Intel Core Ultra 7 155H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, opti

In [ ]:
from ultralytics import YOLO
import supervision as sv
import cv2

# 1. Load your best trained model
# model = YOLO('pothole_project/run_1/weights/best.pt')
model = YOLO('yolo_project/train_v12_small/weights/best.pt')



0: 640x640 (no detections), 363.4ms
Speed: 10.7ms preprocess, 363.4ms inference, 10.4ms postprocess per image at shape (1, 3, 640, 640)
Number of potholes detected: 0
Saved output image!


In [4]:

# 2. Load Image
# image = cv2.imread('test_road.jpg')
# image = cv2.imread('trial/test126.jpg')
# image = cv2.imread('trial/test146.jpg')
image = cv2.imread('trial/testy001.jpg')

# 3. Run YOLO Prediction
results = model(image)[0]

# 4. Convert to Supervision format (The Magic Step)
detections = sv.Detections.from_ultralytics(results)

# --- REQUIREMENT: "HOW MANY POTHOLES?" ---
pothole_count = len(detections)
print(f"Number of potholes detected: {pothole_count}")

# 5. Filter (Optional): Remove low confidence detections
detections = detections[detections.confidence > 0.5]

# 6. Draw Boxes and Labels
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Draw the boxes on the image
annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections)
# Draw the labels (e.g., "Pothole 0.85")
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

# 7. Show or Save
cv2.imwrite('output_with_supervision.jpg', annotated_image)
print("Saved output image!")


0: 256x640 (no detections), 130.3ms
Speed: 18.7ms preprocess, 130.3ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 640)
Number of potholes detected: 0
Saved output image!


### Lower the confidence level and run against the test and validation datasets


In [1]:
# Load the model using the trained weights
from ultralytics import YOLO
import supervision as sv
import cv2

# 1. Load your best trained model
# model = YOLO('pothole_project/run_1/weights/best.pt')
model = YOLO('yolo_project/train_v12_small/weights/best.pt')

In [3]:
# Run Validation
metrics = model.val(
    data='data.yaml', 
    split='val',       # This tells it to use the 'val' folder defined in your yaml
    conf=0.25,
)

print(f"Mean Average Precision (mAP50): {metrics.box.map50}")

Ultralytics 8.3.233  Python-3.12.12 torch-2.9.1+cpu CPU (Intel Core Ultra 7 155H)


YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 3.11.1 MB/s, size: 69.9 KB)
val: Scanning C:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\labels.cache... 248 images, 18 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 248/248 123.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 6.0s/it 1:365.9ss
                   all        248        446      0.754      0.552      0.644      0.367
Speed: 1.1ms preprocess, 350.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to C:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\runs\detect\val
Mean Average Precision (mAP50): 0.6435270432605488


In [4]:
# Run Validation
metrics = model.val(
    data='data.yaml', 
    split='test',       # This tells it to use the 'test' folder defined in your yaml
    conf=0.25,
)

print(f"Mean Average Precision (mAP50): {metrics.box.map50}")

Ultralytics 8.3.233  Python-3.12.12 torch-2.9.1+cpu CPU (Intel Core Ultra 7 155H)
val: Fast image access  (ping: 0.10.0 ms, read: 9.81.6 MB/s, size: 79.3 KB)
val: Scanning C:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\test\labels... 128 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 468.4it/s 0.3ss
val: New cache created: C:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 7.3s/it 58.1s8.2ss
                   all        128        203      0.652      0.522      0.599      0.372
Speed: 1.6ms preprocess, 431.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to C:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\runs\detect\val2
Mean Average Precision (mAP50): 0.598603940887538


In [6]:
# Generate Predictions on Test / Validation Set for each image and the metadata
from ultralytics import YOLO
from pathlib import Path
import supervision as sv
from tqdm import tqdm
import pandas as pd
import numpy as np

# 1. Load your best trained model
# model = YOLO('yolo_project/train_v12_small/weights/best.pt')

# Generator for the results
results = model.predict(
    # source='data/test/images',              # Path to your test images
    source='data/valid/images',              # Path to your test images
    stream=True,                            # Stream results to process them one by one
    conf=0.01,                              # Cast a wide net for predictions (1% confidence)
    # conf=0.25,                                 # Confidence threshold
    # save=False,                                # Whether to save the results
    # save_txt=False,                            # Whether to save results in txt files
    # project='yolo_project/predictions',        # Where to save results
    # name='test_results',                       # Name of the results folder
    # exist_ok=True                              # Overwrite existing results
)

data_list = []
# Process each result
# for result in tqdm(results, desc="Processing predictions"):
for result in results:
    # Each 'result' corresponds to one image's predictions
    image_path = result.path  # Path to the image
    image_name = Path(image_path).name  # Extract image name
    detections = sv.Detections.from_ultralytics(result)  # Convert to Supervision format

    # Before filtering 
    orig_count = len(detections)  # Number of detections
    orig_boxes = detections.xyxy  # Bounding boxes

    # Filtering logic

    # Filter 1: Confidence threshold
    # Only keep detections with confidence > 10% 
    detections = detections[detections.confidence > 0.1]

    # Filter 2: Size threshold (e.g., width and height > 20 pixels)
    # Assume a pothole is of certain minimum area size (e.g. 20x20 pixels = 400 pixels)
    # detections = detections[ detections.area > 400 ]

    # Handle ground truth boxes
    label_path = Path(str(image_path).replace('images', 'labels')).with_suffix('.txt')
    gt_boxes_xyxy = []
    gt_class_ids = []

    if label_path.exists():
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            # Parse YOLO format: class_id center_x center_y width height (normalized)
            if len(parts) >= 5:
                class_id = int(parts[0])
                _, x_center, y_center, width, height = map(float, parts[:5])
                # Convert Normalized YOLO (xywh) to pixel coordinates (xyxy)
                x1 = (x_center - width / 2 ) * width
                y1 = (y_center - height / 2) * height
                x2 = (x_center + width / 2 ) * width
                y2 = (y_center + height / 2) * height
                # Here you can store or process the ground truth boxes as needed
                gt_boxes_xyxy.append([x1, y1, x2, y2])
                gt_class_ids.append(class_id)
    # Convert to Numpy for Supervision
    if gt_boxes_xyxy:
        gt_boxes_np = np.array(gt_boxes_xyxy)
        gt_classes_np = np.array(gt_class_ids)
        
        # Create Supervision Detections object for Ground Truth
        gt_detections = sv.Detections(
            xyxy=gt_boxes_np,
            class_id=gt_classes_np
        )
    else:
        # Handle case with no ground truth labels (empty image)
        gt_detections = sv.Detections.empty()

    gt_count = len(gt_detections)

    # After filtering
    filtered_count = len(detections)  # Number of detections after filtering
    # Here you can process 'detections' as needed

    # Store the data
    data_list.append({
        'image_name': image_name,
        'original_count': orig_count,
        'filtered_count': filtered_count,
        'original_boxes': orig_boxes,
        'filtered_boxes': detections.xyxy,
        'confidences': detections.confidence,
        # Ground Truth Data
        'gt_count': gt_count,
        'gt_boxes': gt_detections.xyxy.tolist() if len(gt_detections) > 0 else [],
    })

# Convert to DataFrame for easier analysis
df = pd.DataFrame(data_list)
csv_filename = 'prediction_results_val.csv'
df.to_csv(csv_filename, index=False)
print(f"Results saved to {Path(csv_filename).absolute()}")

print("Prediction processing complete!")
print(df.head())
# print(df)

# print(f"Image: {image_name}, Detections: {len(detections)}")


image 1/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\images\0106_jpg.rf.94ce9a4d57934a6767648b0de8f32153.jpg: 640x640 (no detections), 222.4ms
image 2/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\images\0122_jpg.rf.ce18c2ebe76966959cc6d314efa8e2a8.jpg: 640x640 13 0s, 153.5ms
image 3/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\images\0126_jpg.rf.e2a886338e6d02e063bcc11c8726ad45.jpg: 640x640 14 0s, 158.8ms
image 4/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\images\0149_jpg.rf.f89e5b8de933145d4d8a6654b5c84db6.jpg: 640x640 3 0s, 156.3ms
image 5/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\valid\images\0151_jpg.rf.a067a83a8a33184931b64fdfc009eb49.jpg: 640x640 7 0s, 146.9ms
image 6/248 c:\DataA\Centennial\2025_Fall\Neural_COMP258\assignments\group_project\yolo\data\val

In [7]:
# Generate Predictions on Validation Set with Visualization and Metadata Logging
from ultralytics import YOLO
from pathlib import Path
import supervision as sv
from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
import os

# === CONFIGURATION ===
# Assumption: Your data structure is standard YOLO format:
# .../data/valid/images/img1.jpg
# .../data/valid/labels/img1.txt

VALID_IMAGES_DIR = 'data/valid/images'
MODEL_PATH = 'yolo_project/train_v12_small/weights/best.pt'

# Output directories
PRED_OUTPUT_DIR = 'runs/vis_predictions_filtered'
GT_OUTPUT_DIR = 'runs/vis_ground_truth'

# Ensure directories exist
os.makedirs(PRED_OUTPUT_DIR, exist_ok=True)
os.makedirs(GT_OUTPUT_DIR, exist_ok=True)

# --- SETUP ---

# 1. Load trained model
print(f"Loading model from {MODEL_PATH}...")
model = YOLO(MODEL_PATH)

# 2. Define Annotators with distinct colors for visual clarity
# Predictions in RED
pred_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, thickness=2)
pred_label_annotator = sv.LabelAnnotator(color=sv.Color.RED, text_scale=0.5, text_thickness=1)

# Ground Truth in GREEN
gt_box_annotator = sv.BoxAnnotator(color=sv.Color.GREEN, thickness=2)
gt_label_annotator = sv.LabelAnnotator(color=sv.Color.GREEN, text_scale=0.5, text_thickness=1)


# 3. Start Inference Generator
# We use a very low confidence here (0.01) to get raw data, then filter later
results_generator = model.predict(
    source=VALID_IMAGES_DIR,
    stream=True,
    conf=0.01,
    verbose=False
)

data_list = []

print(f"Starting processing...")
print(f"Predictions will be saved to: {PRED_OUTPUT_DIR}")
print(f"Ground Truth will be saved to: {GT_OUTPUT_DIR}")

# === MAIN LOOP ===
for result in tqdm(results_generator, desc="Processing Images"):
    
    # A. General Image Info
    image_path = Path(result.path)
    image_name = image_path.name
    
    # Load original image matrix (we will create copies later for drawing)
    original_frame = result.orig_img
    h, w, _ = original_frame.shape

    # ============================================
    # B. PROCESSING PREDICTIONS
    # ============================================
    raw_detections = sv.Detections.from_ultralytics(result)
    pred_orig_count = len(raw_detections)
    
    # --- Filtering Logic ---
    # 1. Confidence Threshold (> 10%)
    filtered_detections = raw_detections[raw_detections.confidence > 0.1]
    # 2. Area Threshold (> 400px)
    filtered_detections = filtered_detections[filtered_detections.area > 400]
    
    pred_filtered_count = len(filtered_detections)

    # ============================================
    # C. PROCESSING GROUND TRUTH
    # ============================================
    # Construct path to corresponding label text file
    label_path = Path(str(image_path).replace('images', 'labels')).with_suffix('.txt')
    
    gt_boxes_xyxy = []
    # We don't necessarily need class IDs for single-class, but good practice to parse
    gt_class_ids = [] 
    
    if label_path.exists():
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        # Parse YOLO TxT format (normalized cx,cy,w,h) to Pixel format (x1,y1,x2,y2)
        for line in lines:
            parts = list(map(float, line.strip().split()))
            if len(parts) >= 5: # Ensure line isn't malformed
                cls_id = int(parts[0])
                cx, cy, bw, bh = parts[1], parts[2], parts[3], parts[4]
                
                x1 = (cx - bw / 2) * w
                y1 = (cy - bh / 2) * h
                x2 = (cx + bw / 2) * w
                y2 = (cy + bh / 2) * h
                
                gt_boxes_xyxy.append([x1, y1, x2, y2])
                gt_class_ids.append(cls_id)

    # Create Supervision Detections object for GT
    if gt_boxes_xyxy:
        gt_detections = sv.Detections(
            xyxy=np.array(gt_boxes_xyxy),
            class_id=np.array(gt_class_ids)
        )
    else:
        gt_detections = sv.Detections.empty()

    gt_count = len(gt_detections)

    # ============================================
    # D. VISUALIZATION AND SAVING (The new part)
    # ============================================
    
    # 1. VISUALIZE PREDICTIONS (Red)
    # Create a fresh copy of the frame so we only draw predictions on it
    pred_frame = original_frame.copy()
    
    # Create labels showing confidence scores
    pred_labels = [f"Pred: {conf:.2f}" for conf in filtered_detections.confidence]
    
    # Annotate
    pred_frame = pred_box_annotator.annotate(scene=pred_frame, detections=filtered_detections)
    pred_frame = pred_label_annotator.annotate(scene=pred_frame, detections=filtered_detections, labels=pred_labels)
    
    # Save to Prediction folder
    cv2.imwrite(os.path.join(PRED_OUTPUT_DIR, image_name), pred_frame)


    # 2. VISUALIZE GROUND TRUTH (Green)
    # Create another fresh copy of the frame
    gt_frame = original_frame.copy()
    
    # Create simple labels indicating GT
    gt_labels_text = ["GT-Pothole" for _ in range(len(gt_detections))]
    
    # Annotate
    gt_frame = gt_box_annotator.annotate(scene=gt_frame, detections=gt_detections)
    gt_frame = gt_label_annotator.annotate(scene=gt_frame, detections=gt_detections, labels=gt_labels_text)

    # Save to Ground Truth folder
    cv2.imwrite(os.path.join(GT_OUTPUT_DIR, image_name), gt_frame)


    # ============================================
    # E. DATA LOGGING
    # ============================================
    data_list.append({
        'filename': image_name,
        # Predictions
        'pred_count_filtered': pred_filtered_count,
        'pred_boxes_filtered': filtered_detections.xyxy.tolist(),
        # Ground Truth
        'gt_count': gt_count,
        'gt_boxes': gt_detections.xyxy.tolist() if gt_count > 0 else [],
        # Metadata for checking filter effectiveness
        'pred_count_raw': pred_orig_count,
    })


# === FINISH ===
# Create DataFrame and save CSV
df = pd.DataFrame(data_list)
csv_filename = 'analysis_pred_vs_gt.csv'
df.to_csv(csv_filename, index=False)

print("\nProcessing Complete!")
print(f"1. Data saved to: {csv_filename}")
print(f"2. Prediction images saved to: {PRED_OUTPUT_DIR} (Look for Red boxes)")
print(f"3. Ground Truth images saved to: {GT_OUTPUT_DIR} (Look for Green boxes)")
print("-" * 30)
print("First 5 rows of data:")
print(df.head())

Loading model from yolo_project/train_v12_small/weights/best.pt...
Starting processing...
Predictions will be saved to: runs/vis_predictions_filtered
Ground Truth will be saved to: runs/vis_ground_truth


Processing Images: 248it [01:03,  3.92it/s]


Processing Complete!
1. Data saved to: analysis_pred_vs_gt.csv
2. Prediction images saved to: runs/vis_predictions_filtered (Look for Red boxes)
3. Ground Truth images saved to: runs/vis_ground_truth (Look for Green boxes)
------------------------------
First 5 rows of data:
                                           filename  pred_count_filtered  \
0  0106_jpg.rf.94ce9a4d57934a6767648b0de8f32153.jpg                    0   
1  0122_jpg.rf.ce18c2ebe76966959cc6d314efa8e2a8.jpg                    3   
2  0126_jpg.rf.e2a886338e6d02e063bcc11c8726ad45.jpg                    2   
3  0149_jpg.rf.f89e5b8de933145d4d8a6654b5c84db6.jpg                    1   
4  0151_jpg.rf.a067a83a8a33184931b64fdfc009eb49.jpg                    3   

                                 pred_boxes_filtered  gt_count  \
0                                                 []         2   
1  [[134.90692138671875, 549.4906005859375, 258.1...         4   
2  [[503.0929870605469, 392.3218994140625, 611.35...         2   
3 

In [8]:
# Performing inference on an image (for demo purposes)
# --- REQUIREMENT: "PERFORM INFERENCE ON AN IMAGE" ---
from ultralytics import YOLO
import supervision as sv
import cv2
from pathlib import Path

confidence_threshold = 0.25
# # 1. Load your best trained model
# model = YOLO('yolo_project/train_v12_small/weights/best.pt')
# 2. Load Image

# Extract the original filename without extension
image_location = 'demo/demo001.jpg'
input_filename = Path(image_location).stem  # Gets 'demo001'

image = cv2.imread(image_location)


# 3. Run YOLO Prediction
results = model(image, conf = confidence_threshold)[0]
# 4. Convert to Supervision format (The Magic Step)
detections = sv.Detections.from_ultralytics(results)
# --- IGNORE ---
pothole_count = len(detections)
print(f"Number of potholes detected: {pothole_count}")
# --- IGNORE ---
# 5. Filter (Optional): Remove low confidence detections
# detections = detections[detections.confidence > confidence_threshold]
# 6. Draw Boxes and Labels
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
# Draw the boxes on the image
annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections)
# Draw the labels (e.g., "Pothole 0.85")
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)
# 7. Show or Save
# Save with the original name in the output
output_path = f'{input_filename}_annotated.jpg'
# cv2.imwrite(f'output_with_supervision.jpg', annotated_image)
cv2.imwrite(output_path, annotated_image)
# print(f"Saved output image: {output_path}")
# print("Saved output image!")

Number of potholes detected: 2


True